# Building the dataset of research papers

The [Entrez](http://biopython.org/DIST/docs/api/Bio.Entrez-module.html) module, a part of the [Biopython](http://biopython.org/) library, will be used to interface with [PubMed](http://www.ncbi.nlm.nih.gov/pubmed).<br>
You can download Biopython from [here](http://biopython.org/wiki/Download).

In this notebook we will be covering several of the steps taken in the [Biopython Tutorial](http://biopython.org/DIST/docs/tutorial/Tutorial.html), specifically in [Chapter 9  Accessing NCBI’s Entrez databases](http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc109).

In [2]:
%pip install biopython

Note: you may need to restart the kernel to use updated packages.


In [3]:
from Bio import Entrez

# NCBI requires you to set your email address to make use of NCBI's E-utilities
Entrez.email = "a.valdestilhas@vu.nl"

The datasets will be saved as serialized Python objects, compressed with bzip2.
Saving/loading them will therefore require the [pickle](http://docs.python.org/3/library/pickle.html) and [bz2](http://docs.python.org/3/library/bz2.html) modules.

In [4]:
import pickle, bz2, os

## EInfo: Obtaining information about the Entrez databases

In [5]:
# accessing extended information about the PubMed database
pubmed = Entrez.read( Entrez.einfo(db="pubmed"), validate=False )[u'DbInfo']

# list of possible search fields for use with ESearch:
search_fields = { f['Name']:f['Description'] for f in pubmed["FieldList"] }

In search_fields, we find 'TIAB' ('Free text associated with Abstract/Title') as a possible search field to use in searches.

In [6]:
search_fields

{'ALL': 'All terms from all searchable fields',
 'UID': 'Unique number assigned to publication',
 'FILT': 'Limits the records',
 'TITL': 'Words in title of publication',
 'MESH': 'Medical Subject Headings assigned to publication',
 'MAJR': 'MeSH terms of major importance to publication',
 'JOUR': 'Journal abbreviation of publication',
 'AFFL': "Author's institutional affiliation and address",
 'ECNO': 'EC number for enzyme or CAS registry number',
 'SUBS': 'CAS chemical name or MEDLINE Substance Name',
 'PDAT': 'Date of publication',
 'EDAT': 'Date publication first accessible through Entrez',
 'VOL': 'Volume number of publication',
 'PAGE': 'Page number(s) of publication',
 'PTYP': 'Type of publication (e.g., review)',
 'LANG': 'Language of publication',
 'ISS': 'Issue number of publication',
 'SUBH': 'Additional specificity for MeSH term',
 'SI': 'Cross-reference from publication to other databases',
 'MHDA': 'Date publication was indexed with MeSH terms',
 'TIAB': 'Free text associa

## ESearch: Searching the Entrez databases

To have a look at the kind of data we get when searching the database, we'll perform a search for papers authored by Haasdijk:

In [7]:
example_authors = ['Haasdijk E']
example_search = Entrez.read( Entrez.esearch( db="pubmed", term=' AND '.join([a+'[AUTH]' for a in example_authors]) ) )
example_search

{'Count': '41', 'RetMax': '20', 'RetStart': '0', 'IdList': ['39325889', '38631313', '33501027', '33501026', '33500899', '29311830', '28513205', '28513201', '28323435', '28140628', '26933487', '24977986', '24901702', '24852945', '24708899', '24252306', '23580075', '23144668', '22174697', '22154920'], 'TranslationSet': [{'From': 'Haasdijk E[AUTH]', 'To': 'haasdijk e[Author]'}], 'QueryTranslation': 'haasdijk e[Author]'}

Note how the result being produced is not in Python's native string format:

In [8]:
type( example_search['IdList'][0] )

Bio.Entrez.Parser.StringElement

The part of the query's result we are most interested in is accessible through

In [9]:
example_ids = [ int(id) for id in example_search['IdList'] ]
print(example_ids)

[39325889, 38631313, 33501027, 33501026, 33500899, 29311830, 28513205, 28513201, 28323435, 28140628, 26933487, 24977986, 24901702, 24852945, 24708899, 24252306, 23580075, 23144668, 22174697, 22154920]


### PubMed IDs dataset

We will now assemble a dataset comprised of research articles containing the given keyword, in either their titles or abstracts.

In [10]:
search_term = 'andre'

In [11]:
Ids_file = 'data2/' + search_term + '_Ids.pkl.bz2'

In [12]:
if os.path.exists( Ids_file ):
    Ids = pickle.load( bz2.BZ2File( Ids_file, 'rb' ) )
else:
    # determine the number of hits for the search term
    search = Entrez.read( Entrez.esearch( db="pubmed", term=search_term+'[TIAB]', retmax=0 ) )
    total = int( search['Count'] )
    
    # `Ids` will be incrementally assembled, by performing multiple queries,
    # each returning at most `retrieve_per_query` entries.
    Ids_str = []
    retrieve_per_query = 10000
    
    for start in range( 0, total, retrieve_per_query ):
        print('Fetching IDs of results [%d,%d]' % ( start, start+retrieve_per_query ) )
        s = Entrez.read( Entrez.esearch( db="pubmed", term=search_term+'[TIAB]', retstart=0, retmax=retrieve_per_query ) )
        Ids_str.extend( s[ u'IdList' ] )
    
    # convert Ids to integers (and ensure that the conversion is reversible)
    Ids = [ int(id) for id in Ids_str ]
    
    for (id_str, id_int) in zip(Ids_str, Ids):
        if str(id_int) != id_str:
            raise Exception('Conversion of PubMed ID %s from string to integer it not reversible.' % id_str )
    
ids_to_remove = [
    29757679, 28155421, 27936261, 27936256, 27885317, 27810869,
    27732103, 27603804, 27570958, 27570955, 27570953, 27570952,
    27570951, 27570950, 27200465, 26966857, 25590542
]

for id_to_remove in ids_to_remove:
    try:
        Ids.remove(id_to_remove)
    except ValueError:
        print(f"ID {id_to_remove} not found in the list.")

    
    # Save list of Ids
    pickle.dump( Ids, bz2.BZ2File( Ids_file, 'wb' ) )
    
total = len( Ids )
print('%d documents contain the search term "%s".' % ( total, search_term ) )

Fetching IDs of results [0,10000]
ID 29757679 not found in the list.
ID 28155421 not found in the list.
ID 27936261 not found in the list.
ID 27936256 not found in the list.
ID 27885317 not found in the list.
ID 27810869 not found in the list.
ID 27732103 not found in the list.
ID 27603804 not found in the list.
ID 27570958 not found in the list.
ID 27570955 not found in the list.
ID 27570953 not found in the list.
ID 27570952 not found in the list.
ID 27570951 not found in the list.
ID 27570950 not found in the list.
ID 27200465 not found in the list.
ID 26966857 not found in the list.
ID 25590542 not found in the list.
1082 documents contain the search term "andre".


Taking a look at what we just retrieved, here are the last 5 elements of the `Ids` list:

In [13]:
Ids[:5]

[39317926, 39241711, 39202126, 39146557, 39142061]

## ESummary: Retrieving summaries from primary IDs

To have a look at the kind of metadata we get from a call to `Entrez.esummary()`, we now fetch the summary of one of Haasdijk's papers (using one of the PubMed IDs we obtained in the previous section:

In [14]:
example_paper = Entrez.read( Entrez.esummary(db="pubmed", id='34648188') )[0]

def print_dict( p ):
    for k,v in p.items():
        print(k)
        print('\t', v)

print_dict(example_paper)

Item
	 []
Id
	 34648188
PubDate
	 2022 Sep
EPubDate
	 2021 Oct 14
Source
	 J Rural Health
AuthorList
	 ['Fletcher TL', 'Amspoker AB', 'Wassef M', 'Hogan JB', 'Helm A', 'Jackson C', 'Jacobs A', 'Shammet R', 'Speicher S', 'Lindsay JA', 'Cloitre M']
LastAuthor
	 Cloitre M
Title
	 Increasing access to care for trauma-exposed rural veterans: A mixed methods outcome evaluation of a web-based skills training program with telehealth-delivered coaching.
Volume
	 38
Issue
	 4
Pages
	 740-747
LangList
	 ['English']
NlmUniqueID
	 8508122
ISSN
	 0890-765X
ESSN
	 1748-0361
PubTypeList
	 ['Journal Article', 'Randomized Controlled Trial']
RecordStatus
	 PubMed - indexed for MEDLINE
PubStatus
	 ppublish+epublish
ArticleIds
	 {'medline': [], 'pubmed': ['34648188'], 'doi': '10.1111/jrh.12628'}
DOI
	 10.1111/jrh.12628
History
	 {'medline': ['2022/09/24 06:00'], 'pubmed': ['2021/10/15 06:00'], 'entrez': '2021/10/14 12:31'}
References
	 []
HasAbstract
	 IntegerElement(1, attributes={})
PmcRefCount
	 Integer

For now, we'll keep just some basic information for each paper: title, list of authors, publication year, and [DOI](https://en.wikipedia.org/wiki/Digital_object_identifier).

In case you are not familiar with the DOI system, know that the paper above can be accessed through the link  `https://www.doi.org/` followed by the paper's DOI.

In [15]:
( example_paper['Title'], example_paper['AuthorList'], int(example_paper['PubDate'][:4]), example_paper['DOI'] )

('Increasing access to care for trauma-exposed rural veterans: A mixed methods outcome evaluation of a web-based skills training program with telehealth-delivered coaching.',
 ['Fletcher TL', 'Amspoker AB', 'Wassef M', 'Hogan JB', 'Helm A', 'Jackson C', 'Jacobs A', 'Shammet R', 'Speicher S', 'Lindsay JA', 'Cloitre M'],
 2022,
 '10.1111/jrh.12628')

### Summaries dataset

We are now ready to assemble a dataset containing the summaries of all the paper `Ids` we previously fetched.

To reduce the memory footprint, and to ensure the saved datasets won't depend on Biopython being installed to be properly loaded, values returned by `Entrez.read()` will be converted to their corresponding native Python types. We start by defining a function for helping with the conversion of strings:

In [16]:
Summaries_file = 'data2/' + search_term + '_Summaries.pkl.bz2'

In [17]:
if os.path.exists( Summaries_file ):
    Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )
else:
    # `Summaries` will be incrementally assembled, by performing multiple queries,
    # each returning at most `retrieve_per_query` entries.
    Summaries = []
    retrieve_per_query = 500
    
    print('Fetching Summaries of results: ')
    for start in range( 0, len(Ids), retrieve_per_query ):
        if (start % 10000 == 0):
            print('')
            print(start, end='')
        else:
            print('.', end='')
        
        # build comma separated string with the ids at indexes [start, start+retrieve_per_query)
        query_ids = ','.join( [ str(id) for id in Ids[ start : start+retrieve_per_query ] ] )
        
        s = Entrez.read( Entrez.esummary( db="pubmed", id=query_ids ) )
        
        # out of the retrieved data, we will keep only a tuple (title, authors, year, DOI), associated with the paper's id.
        # (all values converted to native Python formats)
        for p in s:
            try:
                f = [
                    ( int( p['Id'] ), (
                        str( p['Title'] ),
                        [ str(a) for a in p['AuthorList'] ],
                        int( p['PubDate'][:4] ),                # keeps just the publication year
                        str( p.get('DOI', '') )            # papers for which no DOI is available get an empty string in their place
                        ) )
                    ]
                Summaries.extend( f )
            except ValueError as e:
                print("\nError with ID " + p['Id'] + ": " + str(e))
                print("Manually remove this ID above and re-run code.")
    
    # Save Summaries, as a dictionary indexed by Ids
    Summaries = dict( Summaries )
    
    pickle.dump( Summaries, bz2.BZ2File( Summaries_file, 'wb' ) )

Fetching Summaries of results: 

0..

Let us take a look at the first 3 retrieved summaries:

In [18]:
{ id : Summaries[id] for id in Ids[:3] }

{39317926: ("Early depiction of anterior spinal arteries and veins in André du Laurens's Historia anatomica humani corporis (1600).",
  ['Gailloud P'],
  2024,
  '10.1080/0964704X.2024.2399535'),
 39241711: ('Reversible Phasonic Control of a Quantum Phase Transition in a Quasicrystal.',
  ['Shimasaki T',
   'Bai Y',
   'Kondakci HE',
   'Dotti P',
   'Pagett JE',
   'Dardia AR',
   'Prichard M',
   'Eckardt A',
   'Weld DM'],
  2024,
  '10.1103/PhysRevLett.133.083405'),
 39202126: ('Survival Probability, Particle Imbalance, and Their Relationship in Quadratic Models.',
  ['Hopjan M', 'Vidmar L'],
  2024,
  '10.3390/e26080656')}

## EFetch: Downloading full records from Entrez

`Entrez.efetch()` is the function that will allow us to obtain paper abstracts. Let us start by taking a look at the kind of data it returns when we query PubMed's database.

In [19]:
q = Entrez.read( Entrez.efetch(db="pubmed", id='34648188', retmode="xml") )['PubmedArticle']

`q` is a list, with each member corresponding to a queried id. Because here we only queried for one id, its results are then in `q[0]`.

In [20]:
type(q), len(q)

(list, 1)

1
At `q[0]` we find a dictionary containing two keys, the contents of which we print below.

In [21]:
type(q[0]), q[0].keys()

(Bio.Entrez.Parser.DictionaryElement,
 dict_keys(['MedlineCitation', 'PubmedData']))

In [22]:
print_dict( q[0][ 'PubmedData' ] )

ReferenceList
	 [{'Reference': [{'Citation': "Sadler AG, Booth BM, Mengeling MA, Doebbeling BN. Life span and repeated violence against women during military service: effects on health status and outpatient utilization. J Women's Health. 2004;13(7):799-811."}, {'Citation': 'Scherrer JF, Xian H, Kapp JMK, et\xa0al. Association between exposure to childhood and lifetime traumatic events and lifetime pathological gambling in a twin cohort. J Nerv Ment Dis. 2007;195(1):72-78.'}, {'Citation': 'Katon JG, Lehavot K, Simpson TL, et\xa0al. Adverse childhood experiences, military service, and adult health. Am J Prev Med. 2015;49(4):573-582.'}, {'Citation': 'Koenen KC, Stellman SD, Sommer Jr JF, Stellman JM. Persisting posttraumatic stress disorder symptoms and their relationship to functioning in Vietnam veterans: a 14-year follow-up. J Trauma Stress. 2008;21(1):49-57.'}, {'Citation': 'Kessler RC. Posttraumatic stress disorder: the burden to the individual and to society. J Clin Psychiatry. 2000

The key `'MedlineCitation'` maps into another dictionary. In that dictionary, most of the information is contained under the key `'Article'`. To minimize the clutter, below we show the contents of `'MedlineCitation'` excluding its `'Article'` member, and below that we then show the contents of `'Article'`.

In [23]:
print_dict( { k:v for k,v in q[0][ 'MedlineCitation' ].items() if k!='Article' } )

OtherAbstract
	 []
KeywordList
	 [ListElement([StringElement('access to care', attributes={'MajorTopicYN': 'N'}), StringElement('telemental health', attributes={'MajorTopicYN': 'N'}), StringElement('trauma', attributes={'MajorTopicYN': 'N'}), StringElement('veterans', attributes={'MajorTopicYN': 'N'}), StringElement('web-based treatment', attributes={'MajorTopicYN': 'N'})], attributes={'Owner': 'NOTNLM'})]
InvestigatorList
	 []
GeneralNote
	 []
SpaceFlightMission
	 []
CitationSubset
	 ['IM']
OtherID
	 []
PMID
	 34648188
DateCompleted
	 {'Year': '2022', 'Month': '09', 'Day': '23'}
DateRevised
	 {'Year': '2022', 'Month': '10', 'Day': '03'}
MedlineJournalInfo
	 {'Country': 'England', 'MedlineTA': 'J Rural Health', 'NlmUniqueID': '8508122', 'ISSNLinking': '0890-765X'}
MeshHeadingList
	 [{'QualifierName': [], 'DescriptorName': StringElement('Female', attributes={'UI': 'D005260', 'MajorTopicYN': 'N'})}, {'QualifierName': [], 'DescriptorName': StringElement('Health Services Accessibility', at

In [24]:
print_dict( q[0][ 'MedlineCitation' ][ 'Article' ] )

ArticleDate
	 [DictElement({'Year': '2021', 'Month': '10', 'Day': '14'}, attributes={'DateType': 'Electronic'})]
Language
	 ['eng']
ELocationID
	 [StringElement('10.1111/jrh.12628', attributes={'EIdType': 'doi', 'ValidYN': 'Y'})]
Journal
	 {'ISSN': StringElement('1748-0361', attributes={'IssnType': 'Electronic'}), 'JournalIssue': DictElement({'Volume': '38', 'Issue': '4', 'PubDate': {'Year': '2022', 'Month': 'Sep'}}, attributes={'CitedMedium': 'Internet'}), 'Title': 'The Journal of rural health : official journal of the American Rural Health Association and the National Rural Health Care Association', 'ISOAbbreviation': 'J Rural Health'}
ArticleTitle
	 Increasing access to care for trauma-exposed rural veterans: A mixed methods outcome evaluation of a web-based skills training program with telehealth-delivered coaching.
Pagination
	 {'StartPage': '740', 'EndPage': '747', 'MedlinePgn': '740-747'}
Abstract
	 {'AbstractText': [StringElement('While rural veterans with trauma exposure repor

A paper's abstract can therefore be accessed with:

In [25]:
{ int(q[0]['MedlineCitation']['PMID']) : str(q[0]['MedlineCitation']['Article']['Abstract']['AbstractText'][0]) }

{34648188: 'While rural veterans with trauma exposure report high rates of posttraumatic stress disorder (PTSD), depression, and functional impairment, utilization of health services is low. This pilot study used mixed qualitative and quantitative methods to evaluate the potential benefits of a transdiagnostic web-based skills training program paired with telehealth-delivered coaching to address a range of symptoms and functional difficulties. The study directed substantial outreach efforts to women veterans who had experienced military sexual trauma given their growing representation in the Veterans Healthcare Administration (VHA) and identified need for services.'}

Some of the ids in our dataset refer to books from the [NCBI Bookshelf](http://www.ncbi.nlm.nih.gov/books/), a collection of freely available, downloadable, on-line versions of selected biomedical books. For such ids, `Entrez.efetch()` returns a slightly different structure, where the keys `[u'BookDocument', u'PubmedBookData']` take the place of the `[u'MedlineCitation', u'PubmedData']` keys we saw above.

### Abstracts dataset

We can now assemble a dataset mapping paper ids to their abstracts.

In [26]:
Abstracts_file = 'data2/' + search_term + '_Abstracts.pkl.bz2'

In [27]:
import http.client
from collections import deque
from xml.dom import minidom
import re

def ch(node, childtype):
    return node.getElementsByTagName(childtype)[0]

if os.path.exists( Abstracts_file ):
    Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )
else:
    # `Abstracts` will be incrementally assembled, by performing multiple queries,
    # each returning at most `retrieve_per_query` entries.
    Abstracts = deque()
    retrieve_per_query = 500
    
    print('Fetching Abstracts of results: ')
    for start in range( 0, len(Ids), retrieve_per_query ):
        if (start % 10000 == 0):
            print('')
            print(start, end='')
        else:
            print('.', end='')
        
        # build comma separated string with the ids at indexes [start, start+retrieve_per_query)
        query_ids = ','.join( [ str(id) for id in Ids[ start : start+retrieve_per_query ] ] )
        
        # issue requests to the server, until we get the full amount of data we expect
        while True:
            try:
                #s = Entrez.read( Entrez.efetch(db="pubmed", id=query_ids, retmode="xml" ) )['PubmedArticle']
                s = minidom.parse( Entrez.efetch(db="pubmed", id=query_ids, retmode="xml" ) ).getElementsByTagName("PubmedArticle")
            except http.client.IncompleteRead:
                print('r', end='')
                continue
            break
        
        i = 0
        for p in s:
            abstr = ''
            if (p.getElementsByTagName('MedlineCitation')):
                citNode = ch(p,'MedlineCitation')
                pmid = ch(citNode,'PMID').firstChild.data
                articleNode = ch(citNode,'Article')
                if (articleNode.getElementsByTagName('Abstract')):
                    try:
                        abstr = ch(ch(articleNode,'Abstract'),'AbstractText').firstChild.data
                    except AttributeError:
                        abstr = ch(ch(articleNode,'Abstract'),'AbstractText').toprettyxml("  ")
                        abstr = re.sub(r"\s+", " ", re.sub("<[^>]*>", "", abstr))
            elif (p.getElementsByTagName('BookDocument')):
                bookNode = ch(p,'BookDocument')
                pmid = ch(bookNode,'PMID').firstChild.data
                if (bookNode.getElementsByTagName('Abstract')):
                    try:
                        abstr = ch(ch(bookNode,'Abstract'),'AbstractText').firstChild.data
                    except AttributeError:
                        abstr = ch(ch(bookNode,'Abstract'),'AbstractText').toprettyxml("  ")
                        abstr = re.sub(r"\s+", " ", re.sub("<[^>]*>", "", abstr))
            else:
                raise Exception('Unrecognized record type, for id %d (keys: %s)' % (Ids[start+i], str(p.keys())) )
            
            Abstracts.append( (int(pmid), str(abstr)) )
            i += 1
    
    # Save Abstracts, as a dictionary indexed by Ids
    Abstracts = dict( Abstracts )
    
    pickle.dump( Abstracts, bz2.BZ2File( Abstracts_file, 'wb' ) )

Fetching Abstracts of results: 

0

..

Taking a look at one paper's abstract:

In [28]:
Abstracts[34648188]

KeyError: 34648188

## ELink: Searching for related items in NCBI Entrez

To understand how to obtain paper citations with Entrez, we will first assemble a small set of PubMed IDs, and then query for their citations.
To that end, we search here for papers published in the Nature journal with our given keyword.

In [48]:
CA_search_term = search_term+'[TIAB] AND Nature[JOUR]'
CA_ids = Entrez.read( Entrez.esearch( db="pubmed", term=CA_search_term ) )['IdList']
CA_ids

['36859543', '36418404', '28332550', '27556938', '25592533', '18385673', '17934448', '17410168', '17377536', '17108956', '15635411', '12748612', '9623995', '9333238', '9126732', '9000073', '8893004', '8596627', '7477374', '7990957']

In [49]:
CA_summ = {
    p['Id'] : ( p['Title'], p['AuthorList'], p['PubDate'][:4], p['FullJournalName'], p.get('DOI', '') )
    for p in Entrez.read( Entrez.esummary(db="pubmed", id=','.join( CA_ids )) )
    }
CA_summ

{'36859543': ('Cardiogenic control of affective behavioural state.',
  ['Hsueh B', 'Chen R', 'Jo Y', 'Tang D', 'Raffiee M', 'Kim YS', 'Inoue M', 'Randles S', 'Ramakrishnan C', 'Patel S', 'Kim DK', 'Liu TX', 'Kim SH', 'Tan L', 'Mortazavi L', 'Cordero A', 'Shi J', 'Zhao M', 'Ho TT', 'Crow A', 'Yoo AW', 'Raja C', 'Evans K', 'Bernstein D', 'Zeineh M', 'Goubran M', 'Deisseroth K'],
  '2023',
  'Nature',
  '10.1038/s41586-023-05748-8'),
 '36418404': ('Structured cerebellar connectivity supports resilient pattern separation.',
  ['Nguyen TM', 'Thomas LA', 'Rhoades JL', 'Ricchi I', 'Yuan XC', 'Sheridan A', 'Hildebrand DGC', 'Funke J', 'Regehr WG', 'Lee WA'],
  '2023',
  'Nature',
  '10.1038/s41586-022-05471-w'),
 '28332550': ('Birds of play demonstrate the infectious power of emotion.',
  [],
  '2017',
  'Nature',
  '10.1038/543464a'),
 '27556938': ('Serotonin engages an anxiety and fear-promoting circuit in the extended amygdala.',
  ['Marcinkiewcz CA', 'Mazzone CM', "D'Agostino G", 'Halladay

Because we restricted our search to papers in an open-access journal, you can then follow their DOIs to freely access their PDFs at the journal's website.

We will now issue calls to `Entrez.elink()` using these PubMed IDs, to retrieve the IDs of papers that cite them.
The database from which the IDs will be retrieved is [PubMed Central](http://www.ncbi.nlm.nih.gov/pmc/), a free digital database of full-text scientific literature in the biomedical and life sciences.

A complete list of the kinds of links you can retrieve with `Entrez.elink()` can be found [here](http://eutils.ncbi.nlm.nih.gov/entrez/query/static/entrezlinks.html).

In [50]:
CA_citing = {
    id : Entrez.read( Entrez.elink(
            cmd = "neighbor",               # ELink command mode: "neighbor", returns
                                            #     a set of UIDs in `db` linked to the input UIDs in `dbfrom`.
            dbfrom = "pubmed",              # Database containing the input UIDs: PubMed
            db = "pmc",                     # Database from which to retrieve UIDs: PubMed Central
            LinkName = "pubmed_pmc_refs",   # Name of the Entrez link to retrieve: "pubmed_pmc_refs", gets
                                            #     "Full-text articles in the PubMed Central Database that cite the current articles"
            from_uid = id                   # input UIDs
            ) )
    for id in CA_ids
    }

In [51]:
CA_citing['13907952']

KeyError: '13907952'

We have in `CA_citing[paper_id][0]['LinkSetDb'][0]['Link']` the list of papers citing `paper_id`. To get it as just a list of ids, we can do

In [32]:
cits = [ l['Id'] for l in CA_citing['13907952'][0]['LinkSetDb'][0]['Link'] ]
cits

['6143708', '493438', '369394']

However, one more step is needed, as what we have now are PubMed Central IDs, and not PubMed IDs. Their conversion can be achieved through an additional call to `Entrez.elink()`:

In [33]:
cits_pm = Entrez.read( Entrez.elink( dbfrom="pmc", db="pubmed", LinkName="pmc_pubmed", from_uid=",".join(cits)) )
cits_pm

[{'LinkSetDb': [{'Link': [{'Id': '30258373'}, {'Id': '5443477'}, {'Id': '1996089'}], 'DbTo': 'pubmed', 'LinkName': 'pmc_pubmed'}], 'ERROR': [], 'LinkSetDbHistory': [], 'DbFrom': 'pmc', 'IdList': ['6143708', '493438', '369394']}]

In [34]:
ids_map = { pmc_id : link['Id'] for (pmc_id,link) in zip(cits_pm[0]['IdList'], cits_pm[0]['LinkSetDb'][0]['Link']) }
ids_map

{'6143708': '30258373', '493438': '5443477', '369394': '1996089'}

And to check these papers:

In [35]:
{   p['Id'] : ( p['Title'], p['AuthorList'], p['PubDate'][:4], p['FullJournalName'], p.get('DOI', '') )
    for p in Entrez.read( Entrez.esummary(db="pubmed", id=','.join( ids_map.values() )) )
    }

{'30258373': ('Role of Frontostriatal Connectivity in Adolescents With Excessive Smartphone Use.',
  ['Chun JW', 'Choi J', 'Cho H', 'Choi MR', 'Ahn KJ', 'Choi JS', 'Kim DJ'],
  '2018',
  'Frontiers in psychiatry',
  '10.3389/fpsyt.2018.00437'),
 '5443477': ('A taxonomic study of physiological responses to a psychological stress.',
  ['Bridges PK', 'Jones MT', 'Leak D'],
  '1970',
  'Journal of neurology, neurosurgery, and psychiatry',
  '10.1136/jnnp.33.2.180'),
 '1996089': ('Heat shock transcription factor activates transcription of the yeast metallothionein gene.',
  ['Silar P', 'Butler G', 'Thiele DJ'],
  '1991',
  'Molecular and cellular biology',
  '10.1128/mcb.11.3.1232-1238.1991')}

### Citations dataset

We have now seen all the steps required to assemble a dataset of citations to each of the papers in our dataset.

In [36]:
Citations_file = 'data2/' + search_term + '_Citations.pkl.bz2'
Citations = []

At least one server query will be issued per paper in `Ids`. Because NCBI allows for at most 3 queries per second (see [here](http://biopython.org/DIST/docs/api/Bio.Entrez-pysrc.html#_open)), this dataset will take a long time to assemble. Should you need to interrupt it for some reason, or the connection fail at some point, it is safe to just rerun the cell below until all data is collected.

In [37]:
import http.client

if Citations == [] and os.path.exists( Citations_file ):
    Citations = pickle.load( bz2.BZ2File( Citations_file, 'rb' ) )

if len(Citations) < len(Ids):
    
    i = len(Citations)
    checkpoint = int(len(Ids) / 100) + 1      # save to hard drive at every 1% of Ids fetched
    
    for pm_id in Ids[i:]:               # either starts from index 0, or resumes from where we previously left off
        
        while True:
            try:
                # query for papers archived in PubMed Central that cite the paper with PubMed ID `pm_id`
                c = Entrez.read( Entrez.elink( dbfrom = "pubmed", db="pmc", LinkName = "pubmed_pmc_refs", id=str(pm_id) ) )
                
                c = c[0]['LinkSetDb']
                if len(c) == 0:
                    # no citations found for the current paper
                    c = []
                else:
                    c = [ l['Id'] for l in c[0]['Link'] ]
                    
                    # convert citations from PubMed Central IDs to PubMed IDs
                    p = []
                    retrieve_per_query = 500
                    for start in range( 0, len(c), retrieve_per_query ):
                        query_ids = ','.join( c[start : start+retrieve_per_query] )
                        r = Entrez.read( Entrez.elink( dbfrom="pmc", db="pubmed", LinkName="pmc_pubmed", from_uid=query_ids ) )
                        # select the IDs. If no matching PubMed ID was found, [] is returned instead
                        p.extend( [] if r[0]['LinkSetDb']==[] else [ int(link['Id']) for link in r[0]['LinkSetDb'][0]['Link'] ] )
                    c = p
            
            except http.client.BadStatusLine:
                # Presumably, the server closed the connection before sending a valid response. Retry until we have the data.
                print('r')
                continue
            except HTTPError:
                print('r')
                continue
            break
        
        Citations.append( (pm_id, c) )
        if (i % 10000 == 0):
            print('')
            print(i, end='')
        if (i % 100 == 0):
            print('.', end='')
        i += 1
        
        if i % checkpoint == 0:
            print('\tsaving at checkpoint', i)
            pickle.dump( Citations, bz2.BZ2File( Citations_file, 'wb' ) )
    
    print('\n done.')
    
    # Save Citations, as a dictionary indexed by Ids
    Citations = dict( Citations )
    
    pickle.dump( Citations, bz2.BZ2File( Citations_file, 'wb' ) )

.....	saving at checkpoint 19065
.....	saving at checkpoint 19530
....	saving at checkpoint 19995

20000.....	saving at checkpoint 20460
.....	saving at checkpoint 20925
....	saving at checkpoint 21390
.....	saving at checkpoint 21855
.....	saving at checkpoint 22320
....	saving at checkpoint 22785
.....	saving at checkpoint 23250
.....	saving at checkpoint 23715
....	saving at checkpoint 24180
.....	saving at checkpoint 24645
.....	saving at checkpoint 25110
....	saving at checkpoint 25575
.....	saving at checkpoint 26040
.....	saving at checkpoint 26505
....	saving at checkpoint 26970
.....	saving at checkpoint 27435
....	saving at checkpoint 27900
.....	saving at checkpoint 28365
.....	saving at checkpoint 28830
....	saving at checkpoint 29295
.....	saving at checkpoint 29760
..
30000...	saving at checkpoint 30225
....	saving at checkpoint 30690
.....	saving at checkpoint 31155
.....	saving at checkpoint 31620
....	saving at checkpoint 32085
.....	saving at checkpoint 32550
.....	sa

To see that we have indeed obtained the data we expected, you can match the ids below, with the ids listed at the end of last section.

In [38]:
Citations[13907952]

[30258373, 5443477, 1996089]

## Where do we go from here?

Running the code above generates multiple local files, containing the datasets we'll be working with. Loading them into memory is a matter of just issuing a call like<br>
``data = pickle.load( bz2.BZ2File( data_file, 'rb' ) )``.

The Entrez module will therefore no longer be needed, unless you wish to extend your data processing with additional information retrieved from PubMed.

Should you be interested in looking at alternative ways to handle the data, have a look at the [sqlite3](http://docs.python.org/3/library/sqlite3.html) module included in Python's standard library, or [Pandas](http://pandas.pydata.org/), the Python Data Analysis Library.